In [1]:
import os
import midas_touch2 as mt2
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import stock_mapping as sm
import warnings
import json
import datetime

pd.options.display.max_columns = 100000
warnings.filterwarnings('ignore')

# if not mt5.initialize():
#     print("initialize() failed")
# else:
#     print("MT5 successfully initialised.\n")

In [2]:
current_dir = os.getcwd()
#current_path = os.path.abspath(__file__)
#current_dir = os.path.dirname(current_path)
config_file_name = 'config.json'
config_file_path = os.path.join(current_dir, config_file_name)
with open(config_file_path, 'r') as config_file:
        config = json.load(config_file)

symbols = config['symbols']
api_token = config['api_token']
mapping_file = config['mapping_file_path']
sleep_time = config['sleep_time']
account=config['execute_account']
password=config['password']
server=config['server']
Port_TPSL = config['Port_TPSL']
starting_cash = config['starting_cash']

In [3]:
authorized=mt5.login(account, password=password, server=server)
if authorized:
    print(mt5.account_info())
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))


AccountInfo(login=2117, trade_mode=0, leverage=1, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=1, currency_digits=2, fifo_close=False, balance=999963416.81, credit=0.0, profit=-9.0, equity=999963416.81, margin=2339.1, margin_free=999961077.71, margin_level=42749921.62840409, margin_so_call=50.0, margin_so_so=30.0, margin_initial=2339.1, margin_maintenance=2339.1, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Jeff Ng', server='KIBB-Prod', currency='MYR', company='Kenanga Investment Bank Berhad')


In [4]:
mt5_symbol = sm.get_ticker_list(symbols, file_path=mapping_file)
symbols = [symbol + "SE" for symbol in symbols]
price_df = mt2.download_stocks(symbols, "d", api_token, num_days=20)
price_df = pd.DataFrame(price_df)
price_df.columns = mt5_symbol
weights_df, latest_weights = mt2.run_portfolio(price_df, alpha_n=5)

latest_weights = mt2.generate_random_series(mt5_symbol)
print(f'latest_weights: {latest_weights}')
#ADD DUMMY WEIGHTS HERE TO TEST
total_investment = starting_cash
_, odf_check = mt2.get_opening_orders(mt5_symbol, latest_weights)
folder_name_criteria = ['daily_allocation', 'daily_portfolio']
mt2.delete_folders_if_df_empty(odf_check, folder_name_criteria)
previous_allocation_df = mt2.get_folder(folder_name='daily_allocation')
previous_portfolio_df = mt2.get_folder(folder_name='daily_portfolio')

latest_weights: RHBBANK     0.218852
MAYBANK     0.215981
YNHPROP     0.160222
KPJ         0.064588
MAXIS       0.057743
CDB         0.220405
QL          0.013132
SPRITZER    0.049076
dtype: float64
Total portfolio change position: -0.0669%
The DataFrame is not empty. No folders were deleted.


In [5]:
if previous_allocation_df is not None and previous_portfolio_df is not None:
    prev_holding = previous_allocation_df['Holding Units']
    prev_entry = previous_allocation_df['Entry Units']
    prev_exit = previous_allocation_df['Exit Units']
    total_investment = abs(previous_portfolio_df['Total Value'].iloc[-1])
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    allocation_df['Holding Units'] = prev_holding + prev_entry - prev_exit
else: #first run
    allocation_df = mt2.calculate_stock_allocation(total_investment, latest_weights, price_df)
    print("This is the initial run. No previous allocation data available.")
    allocation_df['Holding Units'] = 0

# Calculate Entry and Exit Units
allocation_df['Entry Units'] = allocation_df.apply(
    lambda row: max(row['Allocated Units'] - row['Holding Units'], 0), axis=1)
allocation_df['Exit Units'] = allocation_df.apply(
    lambda row: max(row['Holding Units'] - row['Allocated Units'], 0), axis=1)
numeric_cols = allocation_df.select_dtypes(include=['number']).astype(float)
for col in numeric_cols.columns:
    allocation_df[col] = numeric_cols[col]
allocation_df

,Share Symbol,Share Price,Allocated Units,Weights,Holding Units,Entry Units,Exit Units
0,RHBBANK,5.45,400.0,0.218852,0.0,400.0,0.0
1,MAYBANK,9.00,200.0,0.215981,200.0,0.0,0.0
2,YNHPROP,4.95,300.0,0.160222,100.0,200.0,0.0
3,KPJ,1.35,400.0,0.064588,500.0,0.0,100.0
4,MAXIS,3.95,100.0,0.057743,500.0,0.0,400.0
5,CDB,4.20,500.0,0.220405,0.0,500.0,0.0
6,QL,5.63,0.0,0.013132,100.0,0.0,100.0
7,SPRITZER,1.81,200.0,0.049076,1200.0,0.0,1000.0


In [7]:
import time
order_ids = mt2.execute_trades_from_data(allocation_df)
time.sleep(60)
orders_list = [mt2.check_order_status(order_id) for order_id in order_ids]
print(f'orders_list: {orders_list}')
deleted_volumes = mt2.delete_orders(orders_list)
print(f'deleted_volumes: {deleted_volumes}')

In [27]:
def get_opening_orders(stock_symbols, weights):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        return None, pd.DataFrame()  # Return an empty DataFrame if MT5 fails to initialize

    all_positions = []
    open_prices = {}
    for symbol in stock_symbols:
        positions = mt5.positions_get(symbol=symbol)
        today_open_price = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_D1, 0, 1)
        if positions is None:
            print(f"No positions for {symbol}, error code={mt5.last_error()}")
        elif len(positions) > 0:
            all_positions.extend(list(positions))
            open_prices[symbol] = today_open_price[0]['open']
    df = pd.DataFrame()
    if all_positions:
        df = pd.DataFrame(all_positions, columns=all_positions[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df = df.drop(['time_msc', 'time_update', 'time_update_msc', 'identifier', 'reason', 
                      'sl', 'tp', 'swap', 'comment', 'external_id'], axis=1)
        df['position_change'] = (df['price_current'] / df['price_open']) - 1
        df['value'] = df['volume'] * df['price_open']
        df['weights'] = df['symbol'].map(weights)
        df['portfolio_change_position'] = df['weights'] * df['position_change']
    total_portfolio_change_position = df['portfolio_change_position'].sum() if not df.empty else 0
    print(f"Total portfolio change position: {total_portfolio_change_position:.4%}")
    return total_portfolio_change_position, df
mt5_symbol = ['RHBBANK', 'MAYBANK', 'YNHPROP', 'KPJ', 'MAXIS', 'CDB', 'QL', 'SPRITZER']
port_change, opening_df = get_opening_orders(mt5_symbol, latest_weights)
opening_df

Total portfolio change position: 0.1852%


,ticket,time,type,magic,volume,price_open,price_current,profit,symbol,position_change,value,weights,portfolio_change_position
0,589169,2023-12-05 17:07:44,0,1116,500.0,5.45,5.48,15.0,RHBBANK,0.005505,2725.0,0.218852,0.001205
1,589171,2023-12-05 17:07:50,0,1116,600.0,4.95,4.97,12.0,YNHPROP,0.004040,2970.0,0.160222,0.000647


In [54]:

def get_filled_orders(symbol_list):
    from_date = datetime.datetime.now()
    to_date = datetime.datetime.now() + datetime.timedelta(days=1)

    print(f"from_date: {from_date}")
    print(f"to_date: {to_date}")
    all_deals = []
    for symbol in symbol_list:
        deals = mt5.history_orders_get(from_date, to_date)
        if deals is None:
            print(f"No deals for {symbol}, error code={mt5.last_error()}")
        elif len(deals) > 0:
            all_deals.extend(list(deals))
    if not all_deals:
        print("No deals found for any of the specified symbols.")
        return pd.DataFrame()
    df = pd.DataFrame(all_deals, columns=all_deals[0]._asdict().keys())
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')
    # Drop columns
    df = df.drop(['time_setup', 'time_done_msc', 'time_setup_msc', 'time_expiration', 'sl',
                  'type_time', 'type_filling', 'reason', 'price_open', 'tp',
                  'position_id', 'position_by_id', 'external_id', 'price_stoplimit'], axis=1)
    df = df.drop_duplicates()
    df['volume_filled'] = df['volume_initial'] - df['volume_current']
    #filter state = 4 and 3
    filled_orders = df[(df['state'] == 4) | (df['state'] == 2)]
    filled_orders = filled_orders[filled_orders['symbol'].isin(symbol_list)]
    return filled_orders

filled_orders = get_filled_orders(mt5_symbol)
filled_orders
################
current_time = datetime.datetime.now()
# Filter for orders done in the last 5 minutes
five_minutes_ago = current_time - datetime.timedelta(minutes=2)
recent_filled_orders = filled_orders[pd.to_datetime(filled_orders['time_done']) >= five_minutes_ago]
recent_filled_orders

from_date: 2023-12-05 17:38:04.160206
to_date: 2023-12-06 17:38:04.160206


,ticket,time_done,type,state,magic,volume_initial,volume_current,price_current,symbol,comment,volume_filled
91,589182,2023-12-05 17:37:47,3,2,1116,300.0,300.0,5.48,RHBBANK,kkyao2 Exit,0.0
92,589183,2023-12-05 17:37:47,2,2,1116,200.0,100.0,9.01,MAYBANK,kkyao2 Entry,100.0
93,589184,2023-12-05 17:37:47,3,2,1116,300.0,300.0,4.97,YNHPROP,kkyao2 Exit,0.0
94,589185,2023-12-05 17:37:26,2,4,1116,600.0,0.0,1.33,KPJ,kkyao2 Entry,600.0
95,589186,2023-12-05 17:37:21,2,4,1116,300.0,0.0,4.23,CDB,kkyao2 Entry,300.0
96,589187,2023-12-05 17:37:47,2,2,1116,200.0,200.0,5.62,QL,kkyao2 Entry,0.0
97,589188,2023-12-05 17:37:47,2,2,1116,700.0,700.0,1.90,SPRITZER,kkyao2 Entry,0.0


In [56]:
buy_order = recent_filled_orders[recent_filled_orders['type'] == 2]
purchase_cost = (buy_order['price_current'] * buy_order['volume_filled']).sum()
print(f'purchase_cost: {purchase_cost}')
sell_order = recent_filled_orders[recent_filled_orders['type'] == 3]
sell_value = (sell_order['price_current'] * sell_order['volume_filled']).sum()
print(f'sell_value: {sell_value}')

cash_balance = 4278 - purchase_cost + sell_value
print(f'cash_balance: {cash_balance}')

purchase_cost: 2968.0
sell_value: 0.0
cash_balance: 1310.0


In [ ]:
allocation_df

In [ ]:
# def close_all_positions(df, comment):
#     for _, row in df.iterrows():
#         symbol = row['symbol']
#         volume = row['volume']
#         # Fetch the current bid price for the symbol
#         symbol_info = mt5.symbol_info_tick(symbol)
#         if symbol_info is None:
#             print(f"Failed to get symbol info for {symbol}, error code =", mt5.last_error())
#             continue
#         current_bid = symbol_info.bid
#         request = {
#             "action": mt5.TRADE_ACTION_DEAL,
#             "symbol": symbol,
#             "volume": volume,
#             "type": mt5.ORDER_TYPE_SELL,
#             "price": current_bid,
#             "deviation": 20,  # Deviation in points
#             "magic": 20231207,       # Magic number, if needed
#             "comment": comment,
#             "type_time": mt5.ORDER_TIME_GTC,
#             "type_filling": mt5.ORDER_FILLING_RETURN,
#         }
#         result = mt5.order_send(request)
#         if result.retcode != mt5.TRADE_RETCODE_DONE:
#             print(f"Failed to close position for {symbol}, error code: {result.retcode}")
#         else:
#             print(f"Position on {symbol} closed successfully.")

# close_all_positions(opening_df, "Closing all positions")

In [ ]:
allocation_df

In [ ]:
if not opening_df.empty:
    print("Opening orders:")
else:
    print("No opening orders found.")
opening_df

In [ ]:
def create_daily_log(filled_orders, opening_df, starting_cash=10000, log_folder="daily_portfolio", log_file_name="balance.csv"):
    log_file_path = os.path.join(log_folder, log_file_name)
    if os.path.exists(log_folder) and os.path.isfile(log_file_path):
        previous_log = pd.read_csv(log_file_path)
        initial_capital = previous_log.iloc[-1]['Cash Balance']
    else:
        initial_capital = starting_cash
    cash_balance = initial_capital
    # Calculate purchase cost
    buy_order = filled_orders[filled_orders['type'] == 2]
    purchase_cost = (buy_order['price_current'] * buy_order['volume_initial']).sum()
    cash_balance -= purchase_cost
    # Calculate portfolio value
    portfolio_value = (opening_df['price_current'] * opening_df['volume']).sum()
    # Calculate sales proceeds
    sell_order = filled_orders[filled_orders['type'] == 3]
    sales_proceed = (sell_order['price_current'] * sell_order['volume_initial']).sum()
    cash_balance += sales_proceed
    # Total value
    total_value = cash_balance + portfolio_value
    # Create the log dataframe
    log_df = pd.DataFrame({
        'Date': [pd.to_datetime('today').date()],
        'Initial Capital': [initial_capital],
        'Cash Balance': [cash_balance],
        'Portfolio Value': [portfolio_value],
        'Purchase Cost': [purchase_cost],
        'Sales Proceeds': [sales_proceed],
        'Total Value': [total_value]
    })
    return log_df
log_df = create_daily_log(recent_filled_orders, opening_df)

mt2.save_df_to_csv(log_df, 
                   folder_name='daily_portfolio', 
                   file_name='balance',
                   append=True)


In [ ]:
log_df

In [ ]:
mt2.save_df_to_csv(allocation_df,
                     folder_name='daily_allocation',
                     file_name='allocation')